In [10]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password =  None #'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Product Sales by popularity

## Description of the use case
This use case involves detecting trends in product sales data using Redis T-Digest. By analyzing the distribution of sales data, businesses can identify patterns and make data-driven decisions.

## Redis Data Structures Used
- **T-Digest**: To detect trends in sales data.
- **Hashes**: To store sales data for each product.


In [27]:
# Mock data for the use case
r.flushdb()
sales_data = {
    '30_days_fd': [5000, 5200, 5300, 5400, 5600,5600,5600,5200,2000,5200,5200,5200,10000, 5300,5200]
 }
   #'1_year_fd': [7700, 7500, 7600, 7800, 7900, 6500, 7000, 7800, 7500,7000,7000],


# Use Hashes to store sales data for each product
for product_id, data in sales_data.items():
    r.hset(product_id, mapping={'sales_data': ','.join(map(str, data))})

# Use T-Digest to detect trends in sales data
for product_id, data in sales_data.items():
    r.tdigest().create(key=f'tdigest:{product_id}')
    for value in data:
        r.execute_command('TDIGEST.ADD', f'tdigest:{product_id}', value)

# Function to get percentiles from T-Digest
def get_percentiles(product_id, percentiles):
    return r.execute_command('TDIGEST.QUANTILE', f'tdigest:{product_id}', *percentiles)

# Detect trends in sales data for each product
# find % of 30_days_fd whose value is above certain threshold 
val = r.tdigest().quantile("tdigest:30_days_fd", 0.75)
print(val)
tmean = r.tdigest().trimmed_mean("tdigest:30_days_fd", 0.2,0.8)
print(tmean)
# Print the sales data for each product
for product_id in sales_data.keys():
    print(f"{product_id}: {r.hget(product_id, 'sales_data')}")


[5600]
5288.888888888889
30_days_fd: 5000,5200,5300,5400,5600,5600,5600,5200,2000,5200,5200,5200,10000,5300,5200
